# Instalando o Spark

In [1]:
!pip install pyspark #==3.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=d396a737b38455b07f5fbe1e46c3c22951b0a8b7cc730220164c454fac157fa7
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2024-02-27 13:16:18--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 54.161.241.46, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  37.9MB/s    in 0.4s    

2024-02-27 13:16:19 (37.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


# Iniciando Sessão Spark

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

# ConfigureSparkUI
conf = SparkConf().set('spark.ui.port', '4050')
sc = SparkContext(conf=conf)
sc.stop()

spark = (
    SparkSession.builder                  # Método da classe que constrói a sessão spark
      .appName("Exercicios Spark")  # Nome do App Spark
      .getOrCreate())                     # Verifica se há uma sessão ativa, e se não há, cria uma nova sessão

In [4]:
get_ipython().system_raw('./ngrok http 4050 &')

In [5]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[],"uri":"/api/tunnels"}


In [6]:
get_ipython().system_raw('./ngrok config add-authtoken 2KBeQEmmd1YNlQ86GGKf3KFOkb3_6sQH7JEnvEhDxwn9A7WnT')


In [9]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, TimestampType


schema_base_pix = StructType([
    StructField('id', IntegerType()),
    StructField('valor', DoubleType()),
    StructField('parte_debitada_nome', StringType()),
    StructField('parte_debitada_conta', StringType()),
    StructField('parte_debitada_banco', StringType()),
    StructField('parte_creditada_nome', StringType()),
    StructField('parte_creditada_conta', StringType()),
    StructField('parte_creditada_banco', StringType()),
    StructField('chave_pix_tipo', StringType()),
    StructField('chave_pix_valor', StringType()),
    StructField('data_transacao', TimestampType())
])

caminho_csv = "/content/drive/MyDrive/Colab Notebooks/Big data/Spark/Aula 13 - Exercícios/base_de_dados.csv"

df = spark.read.csv(
      path=caminho_csv,
      header=True,
      sep=";",
      schema=schema_base_pix,
      timestampFormat="dd/MM/yyyy HH:mm"
)

# Exercícios
1. Descubra a quantidade total de transações na base de dados.
2. Qual foi a quantia total de dinheiro transacional em cada ano?
2. Descubra a quantidade de dinheiro que entrou na Nubank em todo o período.
2. Retorne a quantidade de transações agrupadas por tipo de chave.
3. Descubra qual é a média, mínima e máxima do valor de todas as transações.
4. Quais foram as 5 maiores transações durante todo o período?
5. Para o banco BTG, mostre qual é a chave pix mais utilizada para enviar ou receber transações.
6. Descubra qual foi a maior transação de cada mês na base de dados.
7. Crie uma nova coluna que contenha o valor da transação em dólar.

In [10]:
df.show()

+---+--------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+--------------+---------------+-------------------+
| id|   valor| parte_debitada_nome|parte_debitada_conta|parte_debitada_banco|parte_creditada_nome|parte_creditada_conta|parte_creditada_banco|chave_pix_tipo|chave_pix_valor|     data_transacao|
+---+--------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+--------------+---------------+-------------------+
|  1|    9.93|Dra. Ana Carolina...|            79470453|              Nubank|       Maysa da Cruz|             67162333|                 Itau|           cpf|     8439752610|2022-02-18 13:28:00|
|  2|   15.38|        Ana Caldeira|            19689668|                Itau|        Evelyn Sales|             60005091|             Bradesco|           cpf|    27145380617|2022-04-08 01:47:00|
|  3|   57.58|    Arthur Gonca

In [11]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- valor: double (nullable = true)
 |-- parte_debitada_nome: string (nullable = true)
 |-- parte_debitada_conta: string (nullable = true)
 |-- parte_debitada_banco: string (nullable = true)
 |-- parte_creditada_nome: string (nullable = true)
 |-- parte_creditada_conta: string (nullable = true)
 |-- parte_creditada_banco: string (nullable = true)
 |-- chave_pix_tipo: string (nullable = true)
 |-- chave_pix_valor: string (nullable = true)
 |-- data_transacao: timestamp (nullable = true)



## 1- Descubra a quantidade total de transações na base de dados.


In [15]:
df.count()

100

## 2 - Qual foi a quantia total de dinheiro transacional em cada ano?

In [30]:
from pyspark.sql.functions import year, col

df.groupBy(year('data_transacao')).sum('valor').show()
# Ou
df.withColumn(
    'year', year(col('data_transacao'))
).groupBy('year').sum('valor').show()

+--------------------+------------------+
|year(data_transacao)|        sum(valor)|
+--------------------+------------------+
|                2022|517980.58999999973|
|                2021| 848320.6000000001|
+--------------------+------------------+

+----+------------------+
|year|        sum(valor)|
+----+------------------+
|2022|517980.58999999973|
|2021| 848320.6000000001|
+----+------------------+



## 3 - Descubra a quantidade de dinheiro que entrou na Nubank em todo o período.

In [62]:
df.filter(col('parte_creditada_banco')=='Nubank').groupBy('parte_creditada_banco').sum('valor').show()

from pyspark.sql import functions as F
df.filter(col('parte_creditada_banco') == "Nubank").select(F.sum('valor')).show()

+---------------------+----------+
|parte_creditada_banco|sum(valor)|
+---------------------+----------+
|               Nubank| 302224.64|
+---------------------+----------+

+----------+
|sum(valor)|
+----------+
| 302224.64|
+----------+



## 4 - Retorne a quantidade de transações agrupadas por tipo de chave.

In [63]:
df.groupBy('chave_pix_tipo').count().show()

+--------------+-----+
|chave_pix_tipo|count|
+--------------+-----+
|       celular|   22|
|         email|   29|
|           cpf|   49|
+--------------+-----+



## 5 - Descubra qual é a média, mínima e máxima do valor de todas as transações.

In [72]:
df.select('valor').describe().show()

from pyspark.sql import functions as F

df.select(
    F.max(col('valor')),
    F.avg(col('valor')),
    F.min(col('valor'))
).show()

+-------+------------------+
|summary|             valor|
+-------+------------------+
|  count|               100|
|   mean|13663.011899999998|
| stddev|25715.376055332952|
|    min|              0.03|
|    max|          95977.62|
+-------+------------------+

+----------+------------------+----------+
|max(valor)|        avg(valor)|min(valor)|
+----------+------------------+----------+
|  95977.62|13663.011899999998|      0.03|
+----------+------------------+----------+



## 6 - Quais foram as 5 maiores transações durante todo o período?

In [78]:
df.orderBy(col('valor'), ascending=False).show(5)

+---+--------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+--------------+--------------------+-------------------+
| id|   valor| parte_debitada_nome|parte_debitada_conta|parte_debitada_banco|parte_creditada_nome|parte_creditada_conta|parte_creditada_banco|chave_pix_tipo|     chave_pix_valor|     data_transacao|
+---+--------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+--------------+--------------------+-------------------+
| 54|95977.62|   Henrique Ferreira|             6269315|              Nubank|Sra. Maria Luiza ...|             68542779|                  BTG|           cpf|         14957860211|2021-09-22 10:17:00|
| 86|94736.79|Joao Vitor Cavalc...|            52265189|                 BTG|        Stella Gomes|             93858790|               Nubank|         email|stella.gomes@yaho...|2021-01-26 00:04:00|
| 97|

## 7 - Para o banco BTG, mostre qual é a chave pix mais utilizada para enviar ou receber transações.

In [98]:
df.filter(
    (col('parte_debitada_banco') == "BTG") |
    (col('parte_creditada_banco') == "BTG")
).groupBy('chave_pix_tipo').count().orderBy('count', ascending=False).show()

+--------------+-----+
|chave_pix_tipo|count|
+--------------+-----+
|           cpf|   22|
|         email|   15|
|       celular|   13|
+--------------+-----+

